In [1]:
# Packages
import os #directories
import pandas as pd # dataframe
import re # regular expressions

from bs4 import BeautifulSoup # webscrape
import urllib # webscrape
import requests # webscrape
from urllib.request import urlopen # webscrape
import csv # csv?

filedir = '/Users/Ryan/Desktop/Discord/data'

In [2]:
# set up
url = 'https://animalcrossing.fandom.com/wiki/Bugs_(New_Horizons)#Northern%20Hemisphere'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'html.parser')

In [3]:
# match for critter table
all_matches = soup.find("table",{"class":"sortable"})

In [4]:
# removes html tags
vinfo = BeautifulSoup(str(all_matches)).get_text()

# Lazy edit + split info by line into a list
bugs = vinfo[101:].splitlines()
bugs

['Common butterfly',
 ' ',
 '  160',
 '  Flying',
 ' 4 AM - 7 PM',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' -',
 ' -',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 '',
 '',
 ' Yellow butterfly',
 ' ',
 '  160',
 '  Flying',
 ' 4 AM - 7 PM',
 ' -',
 ' -',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' -',
 ' -',
 ' ✓',
 ' ✓',
 ' -',
 ' -',
 '',
 '',
 ' Tiger butterfly',
 ' ',
 '  240',
 '  Flying',
 ' 4 AM - 7 PM',
 ' -',
 ' -',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' -',
 ' -',
 ' -',
 '',
 '',
 ' Peacock butterfly',
 ' ',
 '  2500',
 '  Flying by Hybrid Flowers',
 ' 4 AM - 7 PM',
 ' -',
 ' -',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' -',
 ' -',
 ' -',
 ' -',
 ' -',
 ' -',
 '',
 '',
 ' Common bluebottle',
 ' ',
 '  300',
 '  Flying',
 ' 4 AM - 7 PM',
 ' -',
 ' -',
 ' -',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' -',
 ' -',
 ' -',
 ' -',
 '',
 '',
 ' Paper kite butterfly',
 ' ',
 '  1000',
 '  Flying',
 ' 8 AM - 7 PM',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 ' ✓',
 '',
 '',
 ' G

In [5]:
# data cleaning
# Remove trailing white space in each item
bugs = [x.strip() for x in bugs]

# remove blank items in bugs list
bugs = [x for x in bugs if x]

In [6]:
# Find all http links for pngs and return a list
pngurl = re.findall(r'https?://[^\s<>"]+|www\.[^\s<>"]+', str(all_matches))

# items were in tuple so combine them
pngurl = [''.join(tups) for tups in pngurl] 

# There are 3 png per insect. Take the first one of every bug
pngurl = pngurl[::3]

In [7]:
# Empty List to append to
rarity = []

# Prepare list of hyperlinks to iterate over
redirect = re.findall(r'/wiki/[\w\s]+[\%27s]?[-]?[\w\s]+', str(all_matches)) # [\%27s allows for apostraphes]

# Go into each critter link page and find the rarity

root = 'https://animalcrossing.fandom.com/'
for x in redirect:
    goto = root + x
    #print(goto)
    response = requests.get(goto)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    
    # Find rarity
    all_matches = soup.find("div",{"data-source":"rarity"}) # find the rarity table at each critter page
    string = BeautifulSoup(str(all_matches)).get_text()
    rarity.append(string.splitlines()[2])

In [8]:
# create lists
names = bugs[::16] # splice: start, end, steps
price = bugs[1::16]
location = bugs[2::16]
time = bugs[3::16]
jan = bugs[4::16]
feb = bugs[5::16]
mar = bugs[6::16]
apr = bugs[7::16]
may = bugs[8::16]
jun = bugs[9::16]
jul = bugs[10::16]
aug = bugs[11::16]
sep = bugs[12::16]
oc = bugs[13::16]
nov = bugs[14::16]
dec = bugs[15::16]

# Create dataframe
dic = {'Name': names,
       'Price': price,
       'Location': location,
       'Time':time,
       'Rarity': rarity,
       'Jan': jan,
       'Feb': feb,
       'Mar': mar,
       'Apr': apr,
       'May': may,
       'Jun': jun,
       'Jul': jul,
       'Aug': aug,
       'Sep': sep,
       'Oct': oc,
       'Nov':nov,
       'Dec':dec,
       'pngurl':pngurl
          }
df=pd.DataFrame.from_dict(dic)

# searchable term
df['search'] = df.Name.apply(lambda x: (x.replace(" ", "")).lower())

In [9]:
# Replaces checks and minus with boolean
def conv(x):
    if x == '✓':
        return True
    elif x == '-':
        return False

# Function to clean Rarity column
def clean(x):
    a = re.findall(r'^[^\(]+',x)
    a = ' '.join(a)
    a = (a.strip()).lower()
    return a

In [10]:
# Fix
df.Jan = df.Jan.apply(lambda x: conv(x))
df.Feb = df.Feb.apply(lambda x: conv(x))
df.Mar = df.Mar.apply(lambda x: conv(x))
df.Apr = df.Apr.apply(lambda x: conv(x))
df.May = df.May.apply(lambda x: conv(x))
df.Jun = df.Jun.apply(lambda x: conv(x))
df.Jul = df.Jul.apply(lambda x: conv(x))
df.Aug = df.Aug.apply(lambda x: conv(x))
df.Sep = df.Sep.apply(lambda x: conv(x))
df.Oct = df.Oct.apply(lambda x: conv(x))
df.Nov = df.Nov.apply(lambda x: conv(x))
df.Dec = df.Dec.apply(lambda x: conv(x))
df.Rarity = df.Rarity.apply(lambda x: clean(x))

In [11]:
df

,Name,Price,Location,Time,Rarity,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,pngurl,search
0,Common butterfly,160,Flying,4 AM - 7 PM,common,True,True,True,True,True,True,False,False,True,True,True,True,https://vignette.wikia.nocookie.net/animalcros...,commonbutterfly
1,Yellow butterfly,160,Flying,4 AM - 7 PM,common,False,False,True,True,True,True,False,False,True,True,False,False,https://vignette.wikia.nocookie.net/animalcros...,yellowbutterfly
2,Tiger butterfly,240,Flying,4 AM - 7 PM,uncommon,False,False,True,True,True,True,True,True,True,False,False,False,https://vignette.wikia.nocookie.net/animalcros...,tigerbutterfly
3,Peacock butterfly,2500,Flying by Hybrid Flowers,4 AM - 7 PM,uncommon,False,False,True,True,True,True,False,False,False,False,False,False,https://vignette.wikia.nocookie.net/animalcros...,peacockbutterfly
4,Common bluebottle,300,Flying,4 AM - 7 PM,common,False,False,False,True,True,True,True,True,False,False,False,False,https://vignette.wikia.nocookie.net/animalcros...,commonbluebottle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Pill bug,250,Hitting Rocks,11 PM - 4 PM,uncommon,True,True,True,True,True,True,False,False,True,True,True,True,https://vignette.wikia.nocookie.net/animalcros...,pillbug
76,Centipede,300,Hitting Rocks,4 PM - 11 PM,scarce,True,True,True,True,True,True,False,False,True,True,True,True,https://vignette.wikia.nocookie.net/animalcros...,centipede
77,Spider,600,Shaking Trees,7 PM - 8 AM,uncommon,True,True,True,True,True,True,True,True,True,True,True,True,https://vignette.wikia.nocookie.net/animalcros...,spider
78,Tarantula,8000,On the Ground,7 PM - 4 AM,rare,True,True,True,True,False,False,False,False,False,False,True,True,https://vignette.wikia.nocookie.net/animalcros...,tarantula


In [12]:
# Write to csv
df.to_csv(filedir + '/bug_data.csv', index = None, header = True)